#### Import the required libraries

In [1]:
import numpy as np
import pandas as pd
import psycopg2 as pg2
from psycopg2 import sql

#### Establish connection to an existing database database


In [2]:
#establishing the connection
conn = pg2.connect(
   database="excercises",
    user='nduser1',
    password='sql@projects',
    host='localhost',
    port= '5432'
)
  
conn.autocommit = True
print('Connection Successful')

cursor = conn.cursor()

Connection Successful


### Create a new database

In [3]:
#create  a cursor 
cursor = conn.cursor()

#Create a database
createDatabase = '''create database hotelbookings'''

cursor.execute(createDatabase)
print('Database created')


Database created


### Now connect to the newly created database

In [8]:
conn2 = pg2.connect(
   database="hotelbookings",
    user='nduser1',
    password='sql@projects',
    host='localhost',
    port= '5432'
)
  
conn2.autocommit = True
cur = conn2.cursor()

print('Connection Successful')


Connection Successful


### Now import the dataset and extract the column names, column data types and dynamically create a table and add the columns

To create a table in the database we need to specify the column names and datatype of each column.

For a high dimensionl dataset (The choosen dataset contains 32 features) it is not possible to enter the column names manually.
 
So I will extract the column names using pandas and then create a table with dynamic column names and datatypes.

### Import the dataset

In [5]:
df = pd.read_csv('hotel_bookings.csv')
df.head(1)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01


#### Extract the column names and column data types as a list

In [9]:
# Extract the column names
cols = list(df.columns.values)
# Extract the column data types as a list
datatypes = list(df.dtypes)
datatypes[2]

dtype('int64')

#### Create a function that will create a table with dynamic columnNames and datatypes

In [12]:
def createTable(name, allcolumns, alldatatypes):
    fields = []
    for i in range(len(allcolumns)):
        columnName = allcolumns[i]
        coldatatype = str(alldatatypes[i])

        #In SQL the datatypes are integer and float
        #But here datatypes are labeled as int64 and float64, so I will  change the string that specifies the data type

        if coldatatype == 'int64':
            coldatatype = 'int'
        elif coldatatype == 'float64':
            coldatatype = 'float'
        else:
            coldatatype = 'varchar'  #for object data types they are changed to varchar later in SQL I will change them with there original data type.
        

        fields.append( sql.SQL( "{} {}" ).format( sql.Identifier(columnName), sql.SQL(coldatatype) ) )

        #This field will contains all columns as (columnName, columnDatatype) tuple

    #Write the query to create the table

    qry = sql.SQL( "CREATE TABLE {tbl_name} ( {columnsAndDatatype} );" ).format(
        tbl_name = sql.Identifier( name ),
        columnsAndDatatype = sql.SQL( ', ' ).join( fields ))

    #Execute the query
    cur.execute(qry)

    return 'Table created and columns are added successfuly'

### Now call the function to create the table

In [13]:
createTable(name='bookings', allcolumns=cols,alldatatypes=datatypes)

'Table created and columns are added successfuly'

#### I just created a database with dynamic table name, column names and data types. Thanks to psycopg2 for this facility.

#### Close all the connections

In [15]:
cursor.close()
cur.close()
conn.close()
conn2.close()

#### Now, to import the data values to the table, We can go to pgadmin and can easily import the csv file using GUI options.
